In [3]:
i = (28,28,1,1)
k = (3,3,6)
s = (1,1)
d = (1,1)

(1, 1)

In [4]:
input_size = i[1:end-2]
kernel_size = _kernelsize(k[1:end-1], d)
fm_size = _featuremapsize(input_size, kernel_size, s)
ws = (k[1:2]...,i[3],k[3])
bs = (1,1,k[3],1)
os = (fm_size...,k[end], i[end])

(26, 26, 6, 1)

In [5]:
w = randn(ws...)

3×3×1×6 Array{Float64,4}:
[:, :, 1, 1] =
 -0.316134  -0.867292  -0.161843
  0.315662   0.151399   0.577581
  1.19151   -2.50737    1.17203 

[:, :, 1, 2] =
  1.20874    0.337586   2.53884 
  0.202205  -0.269008  -0.490257
 -0.523906  -0.296674   1.36464 

[:, :, 1, 3] =
  0.894372  -0.35664   -1.3407  
 -1.14862    0.412791  -0.216177
 -1.51219   -1.97106    0.945647

[:, :, 1, 4] =
 -0.764925  -0.31621     0.881995
 -1.05212    0.0747858  -0.71708 
  0.567681  -2.21042     0.718896

[:, :, 1, 5] =
 -1.73364   0.553788   0.768235
 -0.19523   0.659011   0.525068
  1.14321  -1.48487   -1.90763 

[:, :, 1, 6] =
 1.25747   0.553355  -0.0945626
 1.47351   0.583645  -0.850358 
 0.179956  1.05547    0.540028 

In [1]:
_dimcheck_convolutionkernel(inputsize::Tuple{Int, Int}, kernelsize::Tuple{Int, Int}) = prod(inputsize .> kernelsize) || throw(ArgumentError("Kernel size $kernelsize cannot be larger than Input size $inputsize"))
_kernelsize(kernelsize::Tuple{Int, Int}, dilations::Tuple{Int, Int}) = ((dilations .* kernelsize) .- (dilations.-1))
_featuremapsize(inputsize::Tuple{Int, Int}, kernelsize::Tuple{Int, Int}, strides::Tuple{Int, Int}) = div.(inputsize .- kernelsize, strides) .+ 1
_convolutionkernelmargin(inputsize::Tuple{Int, Int}, kernelsize::Tuple{Int, Int}) = Int.(ceil.(inputsize .% kernelsize))

_convolutionkernelmargin (generic function with 1 method)

In [53]:
"Creates convolution matrix according to given input, kernel, stride and dilation configuration"
function im2col(conv_matrix, conv_kernel, input_size::Tuple{Int, Int}, kernel_size::Tuple{Int, Int}, strides::Tuple{Int, Int}, dilations::Tuple{Int, Int}, fm_size::Tuple{Int, Int})
    margin = input_size.%fm_size
    t1 = zeros(input_size)
    for ci in 1:size(conv_kernel)[4]
        for ki in 1:size(conv_kernel)[3]
            idx = 1
            for i in 1:strides[1]:(fm_size[1]*strides[1])
                for j in 1:strides[2]:(fm_size[2]*strides[2])
                    t1.=0
                    t1[i:dilations[1]:(i+kernel_size[1]-1), j:dilations[2]:(j+kernel_size[2]-1)] = conv_kernel[:,:,ki,ci]'
                    conv_matrix[idx,:,ki,ci] .= t1'[:]
                    idx+=1
                end
            end
        end
    end
end

im2col

In [54]:
conv_mat = zeros(prod(fm_size), prod(input_size), size(w)[3:end]...)

676×784×1×6 Array{Float64,4}:
[:, :, 1, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

In [56]:
@time im2col(conv_mat, w, input_size, kernel_size, s, d, fm_size)

  0.056858 seconds (24.34 k allocations: 25.938 MiB, 38.34% gc time)


In [57]:
"Creates convolution matrix according to given input, kernel, stride and dilation configuration"
function im2col_legacy(weight, input_size::Tuple{Int, Int}, kernel_size::Tuple{Int, Int}, strides::Tuple{Int, Int}, dilations::Tuple{Int, Int}, fm_size::Tuple{Int, Int})
    margin = input_size.%fm_size
    ap=[]
    for i in 1:strides[1]:(fm_size[1]*strides[1])
        for j in 1:strides[2]:(fm_size[2]*strides[2])
            t1 = zeros(input_size)
            t1[i:dilations[1]:(i+kernel_size[1]-1), j:dilations[2]:(j+kernel_size[2]-1)] = weight'
            ap = prod(size(ap) .<= 0) ? t1'[:] : hcat(ap, t1'[:])
        end
    end

    return ap
end

im2col_legacy

In [59]:
@time permutedims(cat([cat([im2col_legacy(w[:,:,ki,ci], input_size, kernel_size, s, d, fm_size) for ki in 1:size(w)[3]]...,dims=3) for ci in 1:size(w)[4]]...,dims=4),(2,1,3,4))

  6.210985 seconds (311.35 k allocations: 8.153 GiB, 19.07% gc time)


676×784×1×6 Array{Float64,4}:
[:, :, 1, 1] =
 -0.316134   0.315662   1.19151    0.0       …   0.0       0.0       0.0    
  0.0       -0.316134   0.315662   1.19151       0.0       0.0       0.0    
  0.0        0.0       -0.316134   0.315662      0.0       0.0       0.0    
  0.0        0.0        0.0       -0.316134      0.0       0.0       0.0    
  0.0        0.0        0.0        0.0           0.0       0.0       0.0    
  0.0        0.0        0.0        0.0       …   0.0       0.0       0.0    
  0.0        0.0        0.0        0.0           0.0       0.0       0.0    
  0.0        0.0        0.0        0.0           0.0       0.0       0.0    
  0.0        0.0        0.0        0.0           0.0       0.0       0.0    
  0.0        0.0        0.0        0.0           0.0       0.0       0.0    
  0.0        0.0        0.0        0.0       …   0.0       0.0       0.0    
  0.0        0.0        0.0        0.0           0.0       0.0       0.0    
  0.0        0.0        0.0    

In [47]:
conv_mat = zeros(prod(fm_size), prod(input_size), size(w)[3:end]...)
im2col(conv_mat, w, input_size, kernel_size, s, d, fm_size)

676×784×1×6 Array{Float64,4}:
[:, :, 1, 1] =
 -0.316134   0.315662   1.19151    0.0       …   0.0       0.0       0.0    
  0.0       -0.316134   0.315662   1.19151       0.0       0.0       0.0    
  0.0        0.0       -0.316134   0.315662      0.0       0.0       0.0    
  0.0        0.0        0.0       -0.316134      0.0       0.0       0.0    
  0.0        0.0        0.0        0.0           0.0       0.0       0.0    
  0.0        0.0        0.0        0.0       …   0.0       0.0       0.0    
  0.0        0.0        0.0        0.0           0.0       0.0       0.0    
  0.0        0.0        0.0        0.0           0.0       0.0       0.0    
  0.0        0.0        0.0        0.0           0.0       0.0       0.0    
  0.0        0.0        0.0        0.0           0.0       0.0       0.0    
  0.0        0.0        0.0        0.0       …   0.0       0.0       0.0    
  0.0        0.0        0.0        0.0           0.0       0.0       0.0    
  0.0        0.0        0.0    

676×784×1×6 Array{Float64,4}:
[:, :, 1, 1] =
 -0.316134   0.315662   1.19151    0.0       …   0.0       0.0       0.0    
  0.0       -0.316134   0.315662   1.19151       0.0       0.0       0.0    
  0.0        0.0       -0.316134   0.315662      0.0       0.0       0.0    
  0.0        0.0        0.0       -0.316134      0.0       0.0       0.0    
  0.0        0.0        0.0        0.0           0.0       0.0       0.0    
  0.0        0.0        0.0        0.0       …   0.0       0.0       0.0    
  0.0        0.0        0.0        0.0           0.0       0.0       0.0    
  0.0        0.0        0.0        0.0           0.0       0.0       0.0    
  0.0        0.0        0.0        0.0           0.0       0.0       0.0    
  0.0        0.0        0.0        0.0           0.0       0.0       0.0    
  0.0        0.0        0.0        0.0       …   0.0       0.0       0.0    
  0.0        0.0        0.0        0.0           0.0       0.0       0.0    
  0.0        0.0        0.0    